In [1]:
import fitz
def pdf_extract(pdf_path):
    pdf_document = fitz.open(pdf_path)
    text = ""
    for page_num in range(pdf_document.page_count):
        page = pdf_document.load_page(page_num)
        text += page.get_text()
    return text
with open('mexico/Mexico_7.txt','r', encoding = 'utf-8', errors = 'ignore') as file:
    mexico_7 = file.read()
with open('mexico/Mexico_8.txt','r', encoding = 'utf-8', errors = 'ignore') as file:
    mexico_8 = file.read()
mex = mexico_7 +" "+ mexico_8
sc = pdf_extract('southernCone/sc_co_7.pdf')+" "+pdf_extract('southernCone/sc_ft_8.pdf')+" "+pdf_extract('southernCone/sc_ft_9.pdf')+" "+pdf_extract('southernCone/sc_nh_8.pdf')+" "+pdf_extract('southernCone/sc_nh_9.pdf')+" "+pdf_extract('southernCone/sc_no_7.pdf')+" "+pdf_extract('southernCone/sc_no_8.pdf')+" "+pdf_extract('southernCone/sc_no_9.pdf')+" "+pdf_extract('southernCone/sc_ps_7.pdf')+" "+pdf_extract('southernCone/sc_ss_7.pdf')

In [2]:
textbooks = [mex, sc]

# Data Cleaning

In [3]:
import re
from langdetect import detect
from langdetect import LangDetectException
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [4]:
def clean_text(post):
    if detect(post) != 'en':
        return ''
    # Make posts lowercase
    post = post.lower()

    # Remove punctuation
    post = re.sub(r'[^a-zA-Z\s]', ' ', post)
    
    # Remove words with repeated letters
    #post = re.sub(r'([a-zA-Z])\1+', r'\1', post)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(post)
    filtered_text = [word for word in word_tokens if word not in stop_words]
    
    return ' '.join(filtered_text)

In [5]:
cleanTextbooks = []
for i in textbooks:
    cleanTextbooks.append(clean_text(i))

In [6]:
#These have spaces, we use these to select the male and female words
menWords = [" man ", " boy ", " male ", " brother ", " father ", " son ", " husband ", " king ", " prince ", " uncle ", " nephew ", " he ", " him ", " his ", " gentleman ", " sir ", " mr. ", " hero ", " lord ", " patriarch ", " men "]
womenWords = [" woman ", " girl ", " female ", " sister ", " mother ", " daughter ", " wife ", " queen ", " princess ", " aunt ", " niece ", " she ", " her ", " hers ", " lady ", " ma'am "," madam ", " mrs. ", " ms. ", " miss ", " heroine ", " dame ", " matriarch ", " women "]
#These have no spaces, so these can be used to throw away words during TFIDF
menWordsNoSpace = ["man", "boy", "male", "brother", "father", "son", "husband", "king", "prince", "uncle", "nephew", "he", "him", "his", "gentleman", "sir", "mr", "hero", "lord", "patriarch", "men"]
womenWordsNoSpace = ["woman", "girl", "female", "sister", "mother", "daughter", "wife", "queen", "princess", "aunt", "niece", "she", "her", "hers", "lady", "ma'am","madam", "mrs", "heroine", "dame", "matriarch", "women"]

In [7]:
menPara = [[] for _ in range(2)]
womenPara = [[] for _ in range(2)]
for i in range(0, 2):
    text = cleanTextbooks[i]
    paragraphs = [text[j:j+100] for j in range(0, len(text), 100)]
    for paragraph in paragraphs:
        for j in menWords:
            if j in paragraph:
                menPara[i].append(paragraph)
        for j in womenWords:
            if j in paragraph:
                womenPara[i].append(paragraph)

# Occurence

In [8]:
men_count = sum(len(sublist) for sublist in menPara)
print(men_count)
women_count = sum(len(sublist) for sublist in womenPara)
print(women_count)

953
914


# Firstness

In [9]:
femaleFirst = []
for sublist in womenPara:  
    for paragraph in sublist:  
        if isinstance(paragraph, str):  
            words = paragraph.split()  
            for i in range(len(words) - 1): 
                if words[i].lower() in womenWordsNoSpace and words[i + 1].lower() in menWordsNoSpace:
                    femaleFirst.append((words[i], words[i + 1]))
print(len(femaleFirst),femaleFirst)

107 [('woman', 'man'), ('woman', 'boy'), ('girl', 'man'), ('wife', 'prince'), ('wife', 'prince'), ('mother', 'son'), ('daughter', 'father'), ('mother', 'son'), ('daughter', 'father'), ('mother', 'brother'), ('mother', 'father'), ('mother', 'men'), ('mother', 'men'), ('mother', 'brother'), ('girl', 'boy'), ('sister', 'son'), ('daughter', 'brother'), ('daughter', 'son'), ('daughter', 'father'), ('sister', 'son'), ('daughter', 'brother'), ('daughter', 'son'), ('daughter', 'father'), ('sister', 'son'), ('daughter', 'brother'), ('daughter', 'son'), ('daughter', 'father'), ('lady', 'uncle'), ('lady', 'uncle'), ('mother', 'uncle'), ('mother', 'father'), ('sister', 'son'), ('girl', 'boy'), ('girl', 'boy'), ('woman', 'man'), ('women', 'men'), ('woman', 'man'), ('women', 'men'), ('mother', 'father'), ('sister', 'brother'), ('mother', 'father'), ('girl', 'boy'), ('daughter', 'father'), ('mother', 'son'), ('wife', 'husband'), ('mother', 'son'), ('wife', 'husband'), ('mother', 'father'), ('sister',

In [10]:
maleFirst = []
for sublist in menPara:  
    for paragraph in sublist:  
        if isinstance(paragraph, str):  
            words = paragraph.split()  
            for i in range(len(words) - 1):  
                if words[i].lower() in menWordsNoSpace and words[i + 1].lower() in womenWordsNoSpace:
                    maleFirst.append((words[i], words[i + 1]))
print(len(maleFirst),maleFirst)

136 [('boy', 'girl'), ('man', 'woman'), ('man', 'woman'), ('man', 'woman'), ('man', 'woman'), ('boy', 'girl'), ('boy', 'girl'), ('king', 'queen'), ('father', 'mrs'), ('brother', 'wife'), ('brother', 'wife'), ('brother', 'wife'), ('brother', 'wife'), ('men', 'woman'), ('men', 'woman'), ('son', 'daughter'), ('son', 'daughter'), ('uncle', 'aunt'), ('brother', 'sister'), ('uncle', 'aunt'), ('brother', 'sister'), ('uncle', 'aunt'), ('brother', 'sister'), ('father', 'mother'), ('father', 'mother'), ('brother', 'sister'), ('brother', 'sister'), ('men', 'women'), ('brother', 'aunt'), ('men', 'women'), ('men', 'women'), ('men', 'women'), ('father', 'sister'), ('father', 'sister'), ('father', 'sister'), ('nephew', 'niece'), ('nephew', 'niece'), ('nephew', 'niece'), ('nephew', 'niece'), ('father', 'sister'), ('brother', 'daughter'), ('son', 'sister'), ('father', 'mother'), ('father', 'sister'), ('brother', 'daughter'), ('son', 'sister'), ('father', 'mother'), ('father', 'sister'), ('brother', 'da

In [11]:
sameLevel = {
    "siblings": {"brother", "sister"},
    "parent_child": {"father", "mother", "son", "daughter"},
    "spouses": {"husband", "wife"},
    "royalty": {"king", "queen", "prince", "princess"},
    "extended_family": {"uncle", "aunt", "nephew", "niece"},
    "personal_pronouns": {"he", "she", "him", "her", "his", "hers"},
    "titles": {"gentleman", "lady", "sir", "ma'am", "mr", "mrs", "madam"},
    "heroic_roles": {"hero", "heroine"},
    "social_roles": {"patriarch", "matriarch"},
    "general_plural": {"men", "women"},
}
flat = {word for group in sameLevel.values() for word in group}
def isSame(pair):
    return pair[0] in flat and pair[1] in flat
femaleFirst = [pair for pair in femaleFirst if isSame(pair)]

In [12]:
maleFirst = [pair for pair in maleFirst if isSame(pair)]
print(len(maleFirst))
print(len(femaleFirst))

94
87


# Sentiment analysis

In [13]:
fm = ', '.join(map(str, [item for sublist in menPara for item in sublist]))
fw = ', '.join(map(str, [item for sublist in womenPara for item in sublist]))
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
fscores = analyzer.polarity_scores(fw)
mscores = analyzer.polarity_scores(fm)
print(fscores,mscores)

{'neg': 0.061, 'neu': 0.786, 'pos': 0.153, 'compound': 1.0} {'neg': 0.054, 'neu': 0.798, 'pos': 0.148, 'compound': 1.0}


# TFIDF

In [14]:
from langdetect import detect
from langdetect import LangDetectException
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

In [15]:
#stemming the words
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
def stem_nested_list(nested_list):
    return [
        [
            ' '.join(stemmer.stem(word) for word in sentence.split())
            for sentence in sublist
        ]
        for sublist in nested_list
    ]
menParaOriginal = menPara
womenParaOriginal = womenPara
menPara = stem_nested_list(menPara)
womenPara = stem_nested_list(womenPara)
fm = ', '.join(map(str, [item for sublist in menPara for item in sublist]))
fw = ', '.join(map(str, [item for sublist in womenPara for item in sublist]))

In [16]:
tobeTFIDF = [" ".join(" ".join(row) for row in menPara)," ".join(" ".join(row) for row in womenPara)]
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(tobeTFIDF)
feature_names = vectorizer.get_feature_names_out()
dense_tfidf = tfidf_matrix.toarray()

In [17]:
#tfidf of female words
doc_index = 1
doc_tfidf_scores = dense_tfidf[doc_index]
word_score_pairs = list(zip(feature_names, doc_tfidf_scores))
sorted_word_scores = sorted(word_score_pairs, key=lambda x: x[1], reverse=True)
top_300_words_fs = sorted_word_scores[:300]
top_300_words_f = [word for word, score in sorted_word_scores[:300]]

In [18]:
#tfidf of male words
doc_index = 0  
doc_tfidf_scores = dense_tfidf[doc_index]
word_score_pairs_m = list(zip(feature_names, doc_tfidf_scores))
sorted_word_scores_m = sorted(word_score_pairs_m, key=lambda x: x[1], reverse=True)
top_300_words_ms = sorted_word_scores_m[:300]
top_300_words_m = [word for word, score in sorted_word_scores_m[:300]]

In [19]:
#remove common words and gendered keywords
common = list(set(top_300_words_f) & set(top_300_words_m))
top_300_words_f = [word for word in top_300_words_f if word not in common]
top_300_words_f = [word for word in top_300_words_f if word not in womenWordsNoSpace]
female_300 = " ".join(top_300_words_f)
top_300_words_m = [word for word in top_300_words_m if word not in common]
top_300_words_m = [word for word in top_300_words_m if word not in menWordsNoSpace]
male_300 = " ".join(top_300_words_m)

In [20]:
print(female_300)
print(male_300)

verb visit tv parti stay femal london song yesterday adam becam french laura ms ng run bishop bore cat elizabeth grammar much niec today whose form star usual dinner gabriela ghost ok possess se vocabulari sue alway band beauti cold feel jack jean ladi mobil monday sit tire afraid ago also american chri co cours differ green happi interest mall mari member part restaur si usa adj america bedroom better britain convers de earli ever fisherman gave hen holiday jami
richard kill robin sheriff dorian studi ar group hot polic episod ten toni twenti box dictionari favourit morion wash away en noun ond order toke alfr da feminin inin masculin inform meet peter robert sea sorri st wo wont would friar tuck aid buy describ footbal hamlet happen la learn na radio somebodi stop tree william becom bill bo citi colleg fals game ing jane note open prison sound street swim sybil huntingdon om sherwood angri bank


In [21]:
from collections import Counter
import re
mwords = male_300.split()
filteredMwords = [word for word in mwords if word in fm]
wordMcounts = Counter(filteredMwords)
topMwords = wordMcounts.most_common(40)

In [22]:
fwords = female_300.split()
filteredFwords = [word for word in fwords if word in fw]
wordFcounts = Counter(filteredFwords)
topFwords = wordFcounts.most_common(40)

In [23]:
from collections import Counter
import re

# Example word lists (assuming male_300 and female_300 are defined as strings)
mwords = male_300.split()  # Male words
fwords = female_300.split()  # Female words

# Filtered words for male
filteredMwords = [word for word in mwords if word in fm]
wordMcounts = Counter(filteredMwords)
topMwords = wordMcounts.most_common(40)
print("Top 40 Male Words:", topMwords)

# Filtered words for female
filteredFwords = [word for word in fwords if word in fw]
wordFcounts = Counter(filteredFwords)
topFwords = wordFcounts.most_common(40)
print("Top 40 Female Words:", topFwords)


Top 40 Male Words: [('richard', 1), ('kill', 1), ('robin', 1), ('sheriff', 1), ('dorian', 1), ('studi', 1), ('ar', 1), ('group', 1), ('hot', 1), ('polic', 1), ('episod', 1), ('ten', 1), ('toni', 1), ('twenti', 1), ('box', 1), ('dictionari', 1), ('favourit', 1), ('morion', 1), ('wash', 1), ('away', 1), ('en', 1), ('noun', 1), ('ond', 1), ('order', 1), ('toke', 1), ('alfr', 1), ('da', 1), ('feminin', 1), ('inin', 1), ('masculin', 1), ('inform', 1), ('meet', 1), ('peter', 1), ('robert', 1), ('sea', 1), ('sorri', 1), ('st', 1), ('wo', 1), ('wont', 1), ('would', 1)]
Top 40 Female Words: [('verb', 1), ('visit', 1), ('tv', 1), ('parti', 1), ('stay', 1), ('femal', 1), ('london', 1), ('song', 1), ('yesterday', 1), ('adam', 1), ('becam', 1), ('french', 1), ('laura', 1), ('ms', 1), ('ng', 1), ('run', 1), ('bishop', 1), ('bore', 1), ('cat', 1), ('elizabeth', 1), ('grammar', 1), ('much', 1), ('niec', 1), ('today', 1), ('whose', 1), ('form', 1), ('star', 1), ('usual', 1), ('dinner', 1), ('gabriela',

# random forest

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd
male_segments = fm.split(" ")  
female_segments = fw.split(" ")  
texts = male_segments + female_segments  
labels = [0] * len(male_segments) + [1] * len(female_segments)  
vectorizer = TfidfVectorizer()  
X = vectorizer.fit_transform(texts)  
feature_names = vectorizer.get_feature_names_out()

clf = RandomForestClassifier(random_state=42, n_estimators=100)
clf.fit(X, labels)


proba = clf.predict_proba(X)[:, 1]  # Probability of being female
female_contributions = np.dot(X.T.toarray(), proba)
female_importance = female_contributions / np.sum(X.toarray(), axis=0)
female_importance_df = pd.DataFrame({
    "Word": feature_names,
    "FemaleImportance": female_importance
}).sort_values(by="FemaleImportance", ascending=False)

print("Top words most related to Female:")
print(female_importance_df.head(10))

Top words most related to Female:
           Word  FemaleImportance
4404      susan          1.000000
4373        sue          1.000000
4180      smith          1.000000
3243      organ          1.000000
2764     meadow          1.000000
454      bishop          1.000000
373       beatl          1.000000
2994  neighbour          1.000000
3962       save          1.000000
743       chink          0.994466


In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd
male_segments = fm.split(" ")  
female_segments = fw.split(" ")  
texts = male_segments + female_segments  
labels = [0] * len(male_segments) + [1] * len(female_segments)  
vectorizer = TfidfVectorizer()  
X = vectorizer.fit_transform(texts)  
feature_names = vectorizer.get_feature_names_out()

clf = RandomForestClassifier(random_state=42, n_estimators=100)
clf.fit(X, labels)

proba = clf.predict_proba(X)[:, 0]  # Probability of being male (class 0)


male_contributions = np.dot(X.T.toarray(), proba)
male_importance = male_contributions / np.sum(X.toarray(), axis=0)
male_importance_df = pd.DataFrame({
    "Word": feature_names,
    "MaleImportance": male_importance
}).sort_values(by="MaleImportance", ascending=False)

print("Top words most related to Male:")
print(male_importance_df.head(10))

Top words most related to Male:
            Word  MaleImportance
4555    thursday             1.0
1366      episod             1.0
4069     sheriff             1.0
1653       friar             1.0
5075      writer             1.0
3649        race             1.0
4072    sherwood             1.0
2711   mannequin             1.0
1520     feminin             1.0
2066  huntingdon             1.0


In [26]:
fm = ', '.join(map(str, [item for sublist in menParaOriginal for item in sublist]))
fw = ', '.join(map(str, [item for sublist in womenParaOriginal for item in sublist]))
ps = PorterStemmer()
words = word_tokenize(fw)
matching = [word for word in words if ps.stem(word) in {"cass", "aborigin","sungen"}]
matching

[]